In [5]:
import joblib
import pandas as pd 
import numpy as np 

In [7]:
modelfile = 'NN_MJrec.pkl.zip'
tfidffile = 'tfidf.pkl'
nn = joblib.load(modelfile)
tfidf = joblib.load(tfidffile)

In [8]:
# made a compressed version of the original pkl file
# joblib.dump(nn, 'NN_MJrec.pkl.zip', compress=5)

In [9]:
text = 'testing string'
text = pd.Series(text)
vect = tfidf.transform(text)

In [13]:
vectdf = pd.DataFrame(vect.todense())
vectdf

,0,1,2,3,4,5,6,7,8,9,...,8657,8658,8659,8660,8661,8662,8663,8664,8665,8666
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# some feature of this model seems to be deprecated, making model from scratch now
# nn.kneighbors([vectdf][0], n_neighbors=5)[1][0].tolist()

AttributeError: 'NearestNeighbors' object has no attribute 'n_samples_fit_'

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

import spacy
from spacy.tokenizer import Tokenizer
from nltk.stem import PorterStemmer
nlp = spacy.load("en_core_web_lg")

In [6]:
df = pd.read_csv('cannabis-strains.zip')
df.head()

,Strain,Type,Rating,Effects,Flavor,Description
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."


In [7]:
df = df.fillna('None')

In [8]:
df['bag_of_words'] = df['Strain']+" "+df["Effects"] +" "+ df["Flavor"] +" "+ df['Description'] +" "+ df['Type']

In [9]:
df['bag_of_words'].head()

0    100-Og Creative,Energetic,Tingly,Euphoric,Rela...
1    98-White-Widow Relaxed,Aroused,Creative,Happy,...
2    1024 Uplifted,Happy,Relaxed,Energetic,Creative...
3    13-Dawgs Tingly,Creative,Hungry,Relaxed,Uplift...
4    24K-Gold Happy,Relaxed,Euphoric,Uplifted,Talka...
Name: bag_of_words, dtype: object

In [10]:
tokenizer = Tokenizer(nlp.vocab)

In [11]:
tokens = []

""" Make them tokens """
for doc in tokenizer.pipe(df['bag_of_words'], batch_size=500):
    doc_tokens = [token.text for token in doc]
    tokens.append(doc_tokens)

df['tokens'] = tokens

In [12]:
df['tokens'].head()

0    [100-Og, Creative,Energetic,Tingly,Euphoric,Re...
1    [98-White-Widow, Relaxed,Aroused,Creative,Happ...
2    [1024, Uplifted,Happy,Relaxed,Energetic,Creati...
3    [13-Dawgs, Tingly,Creative,Hungry,Relaxed,Upli...
4    [24K-Gold, Happy,Relaxed,Euphoric,Uplifted,Tal...
Name: tokens, dtype: object

In [13]:
df.to_csv('cannabis-strains-token.zip')

In [14]:
df = pd.read_csv('cannabis-strains-token.zip')
df.head()

,Unnamed: 0,Strain,Type,Rating,Effects,Flavor,Description,bag_of_words,tokens
0,0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...,"100-Og Creative,Energetic,Tingly,Euphoric,Rela...","['100-Og', 'Creative,Energetic,Tingly,Euphoric..."
1,1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...,"98-White-Widow Relaxed,Aroused,Creative,Happy,...","['98-White-Widow', 'Relaxed,Aroused,Creative,H..."
2,2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...,"1024 Uplifted,Happy,Relaxed,Energetic,Creative...","['1024', 'Uplifted,Happy,Relaxed,Energetic,Cre..."
3,3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...,"13-Dawgs Tingly,Creative,Hungry,Relaxed,Uplift...","['13-Dawgs', 'Tingly,Creative,Hungry,Relaxed,U..."
4,4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%...","24K-Gold Happy,Relaxed,Euphoric,Uplifted,Talka...","['24K-Gold', 'Happy,Relaxed,Euphoric,Uplifted,..."


In [37]:
# if '100-Og' in df['Strain']:
temp = df[df['Strain']=='100-Og'] 
temp.tokens[0]

"['100-Og', 'Creative,Energetic,Tingly,Euphoric,Relaxed', 'Earthy,Sweet,Citrus', '$100', 'OG', 'is', 'a', '50/50', 'hybrid', 'strain', 'that', 'packs', 'a', 'strong', 'punch.', 'The', 'name', 'supposedly', 'refers', 'to', 'both', 'its', 'strength', 'and', 'high', 'price', 'when', 'it', 'first', 'started', 'showing', 'up', 'in', 'Hollywood.', 'As', 'a', 'plant,', '$100', 'OG', 'tends', 'to', 'produce', 'large', 'dark', 'green', 'buds', 'with', 'few', 'stems.', 'Users', 'report', 'a', 'strong', 'body', 'effect', 'of', 'an', 'indica', 'for', 'pain', 'relief', 'with', 'the', 'more', 'alert,', 'cerebral', 'feeling', 'thanks', 'to', 'its', 'sativa', 'side.', 'hybrid']"

In [22]:
df.Strain[0]

'100-Og'

In [42]:

from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words='english')

# Create a vocabulary and get word counts per document
#Similar to fit_predict
dtm = tfidf.fit_transform(df['bag_of_words'])

# Print word counts

# Get feature names to use as dataframe column headers
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())

# View Feature Matrix as DataFrame
dtm.head()

,00,001,07,09,10,100,1024,10th,11,115,...,zeta,zeus,zingers,zion,zipping,zkittlez,zombie,zone,zoning,zoom
0,0.0,0.0,0.0,0.0,0.0,0.559137,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.585883,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:

# Instantiate
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer


# Fit on TF-IDF Vectors
nn  = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [44]:
# Query Using kneighbors 
nn.kneighbors([dtm.iloc[378]])

(array([[0.        , 1.04504707, 1.09588259, 1.26568839, 1.27157659]]),
 array([[378,  90, 929, 888, 918]]))

In [45]:
df['bag_of_words'][378][:200]

'Bright-Moments None None Bright Moments by Gage Green Group is a delicious cross of heirloom Grape Stomper genetics. Created by crossing Grape Stomper and Grape Stomper OG, Gage Green fashions an incr'

In [46]:
df['bag_of_words'][929][:200]

"Sour-Grapes Happy,Relaxed,Uplifted,Euphoric,Hungry Grape,Sweet,Berry Also called 'Sour Grapes,' Grape Stomper\xa0is a craft hybridstrain created by Gage Green Seeds. A complex cross between breeder JojoR"

In [47]:
joblib.dump(nn, 'NN_MJrec.pkl.zip', compress=5)

['NN_MJrec.pkl.zip']

In [48]:
joblib.dump(tfidf, "tfidf.pkl.zip", compress=5)

['tfidf.pkl.zip']

In [49]:
nn = joblib.load('NN_MJrec.pkl.zip')
tfidf = joblib.load('tfidf.pkl.zip')

In [50]:
def recommend(text):
   # Transform
    text = pd.Series(text)
    vect = tfidf.transform(text)

    # Send to df
    vectdf = pd.DataFrame(vect.todense())
    

    # Return a list of indexes
    top5 = nn.kneighbors([vectdf][0], n_neighbors=5)[1][0].tolist()
   
    
    # Send recomendations to DataFrame
    recommendations_df = df.iloc[top5]
    recommendations_df['index']= recommendations_df.index
    
    return recommendations_df

In [51]:
recommend(text)

,Strain,Type,Rating,Effects,Flavor,Description,bag_of_words,tokens,index
1438,Mt-Hood-Magic,hybrid,4.7,"Euphoric,Happy,Creative,Focused,Uplifted","Earthy,Sweet,Citrus","Thought to be exclusive to Portland, Oregon di...","Mt-Hood-Magic Euphoric,Happy,Creative,Focused,...","[Mt-Hood-Magic, Euphoric,Happy,Creative,Focuse...",1438
1273,Lemon-Pie,sativa,4.2,"Energetic,Euphoric,Uplifted,Talkative,Happy","Lemon,Citrus,Diesel",Lemon Pie from Leafs by Snoop is a sativa-domi...,"Lemon-Pie Energetic,Euphoric,Uplifted,Talkativ...","[Lemon-Pie, Energetic,Euphoric,Uplifted,Talkat...",1273
1705,Purple-Og-Kush,indica,4.4,"Sleepy,Relaxed,Happy,Euphoric,Hungry","Earthy,Sweet,Pungent",Have some snacks ready before testing out this...,"Purple-Og-Kush Sleepy,Relaxed,Happy,Euphoric,H...","[Purple-Og-Kush, Sleepy,Relaxed,Happy,Euphoric...",1705
1329,Magic-Jordan,hybrid,4.0,"Euphoric,Focused,Tingly,Uplifted,Creative","Minty,Orange,Lemon",Magic Jordan is another mysterious Hawaiian cu...,"Magic-Jordan Euphoric,Focused,Tingly,Uplifted,...","[Magic-Jordan, Euphoric,Focused,Tingly,Uplifte...",1329
890,Golden-Lemon,hybrid,4.7,"Uplifted,Relaxed,Euphoric,Tingly,Happy","Lemon,Citrus,Sweet",Golden Lemon by DNA Genetics is citrusy strain...,"Golden-Lemon Uplifted,Relaxed,Euphoric,Tingly,...","[Golden-Lemon, Uplifted,Relaxed,Euphoric,Tingl...",890
